# Advent of code 2024
## Challenge 16

This challenge has been very difficult for me. I tried for a while on my own, first trying a recursive and later a queue based algorithm. The problem with my algorithms is that they were trying every possible path, and so it was never finishing. I later tried to improve it by keeping track of the path with the least total points, but it also was not enough. 

I was also unsure which path searching algorithm to use so I looked that up. I almost went for an A* algorithm. But some other users pointed towards a simpler djikstra, even bfs. I looked a bit into Djikstra but then later went for a BFS algorithm based on what I found from others online. 

I had read that the algorithm should prioritise every step in which it goes forward, where it does not turn. This did not work, even when I started keeping track of the shortest path in the right way. 

Implementing BFS in code was also something difficult for me, so I looked online. The first source that I found is this article: https://medium.com/@davidlfliang/intro-graph-theory-in-python-maze-pathfinder-bb65ba8f3833. Which I did implement. I was struggling and it was not going great in my implementation. So I looked for more resource. And I found this solution: https://github.com/fivard/AOC-2024/tree/master/day16 on this Reddit thread: https://www.reddit.com/r/adventofcode/comments/1hfboft/2024_day_16_solutions/.

That last ressource is what I essentially used. I already had a bunch of code written before I found this resource, so I tried to modify my code to achieve the same result as in the resource. But I was still not managing and I could not know what I was doing wrong. 

So what I did is run the solution algorithm on my dataset, which gave me the right answer. I now knew which answer I had to get to. I still was not figuring it out. So in the end, I put the solution algorithm side by side with mine, and I went through both, making modifications to mine accordingly. Then I finally figured it out. 

In short, the solution algorithm is a completely normal BFS, but we keep track of the score of each formed path at every position on the map. If the position on the map has a lower score then the current path, it means a better path was there, and so we stop the current path. 

For part 2, I struggled in the same way. I went with the resource solution. And I tried on my own to implement a reverse pathfinding algorithm after getting the best path. I was not figuring out how to only keep track of the positions, or the paths, that would form a best path scenario. And basing it on the total score that I need to reach was too long. The algorithm based on that was not finishing. 

So I again looked at the solution algorithm. I also ran it on my data set and obtained the answer, now I knew what was the answer I needed to obtain. But then looking at the solution algorithm, the idea is to again use the socre by position. When walking backwards, we keep a patch going and adding its position to the total only if when they step on a position, their score is the same or lower then the saved score for that position, which is the best possible after running part 1. This also allows to walk on positions that overwrote the score of a best path scenario, but was then dropped later on. Such as this:

                                        5005  5006  5007  4013  5009  5010
                                                          4012
                                                          4011

## Part 1
### https://adventofcode.com/2024/day/16

### Finding starting and ending position

In [1]:
maze = []

input_file = open("challenge_16_input.txt", "r")

for line in input_file:
    input_data = list(line.strip())
    maze.append(input_data)

In [2]:
for main_index,line in enumerate(maze):
    for nested_index, value in enumerate(line):
        if value == 'S':
            start = (main_index,nested_index)
        elif value == 'E':
            end = (main_index,nested_index)

In [3]:
# This function provides the allowed moves and corresponding score based on
# given direction
def find_steps(direction):
    if direction == (0,1):
        return [(-1,0,1001),(1,0,1001),(0,1,1)]
    elif direction == (0,-1):
        return [(-1,0,1001),(1,0,1001),(0,-1,1)]
    elif direction == (1,0):
        return [(0,-1,1001),(0,1,1001),(1,0,1)]
    elif direction == (-1,0):
        return [(0,-1,1001),(0,1,1001),(-1,0,1)]

### BFS pathfinding

In [4]:
from collections import deque

def bfs_pathfinding(maze, start, direction, end):
    
    rows = len(maze)
    cols = len(maze[0])
    # we create a main queue of steps to take and initiate with the starting position
    queue = deque([[(start[0],start[1]),(direction[0],direction[1]), 0]])
    # Keeps track of the best score at any one position of the maze
    score_by_position = {(start[0],start[1]): 0}
        
    # We start going through the maze
    while queue:
        
        # We extract the relevant information for the position at which we are now
        current_position_direction_score = queue.popleft()
        current_position = current_position_direction_score[0]
        current_direction = current_position_direction_score[1]
        current_score = current_position_direction_score[2]
         
        r, c = current_position
        cdr, cdc = current_direction
        
        # We extract our allowed steps 
        steps = find_steps(current_direction)
        
        # We check if we can take the step. Especially if the step would provide a better score then a previous path
        # that has already been on that position
        if 0 <= r < rows and 0 <= c < cols and maze[r][c] != '#':
            for dr, dc, score in steps:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] > current_score + score):
                    queue.append([(nr, nc),(dr,dc), current_score + score])
                    score_by_position[(nr, nc)] =  current_score + score
                
    return score_by_position[(end[0],end[1])]

In [ ]:
total_score = bfs_pathfinding(maze, start, (0,1), end)

print(f"Best score: {total_score}")

## Part 2

In [6]:
def bfs_pathfinding_second(maze, start, direction, end):
    # The first walk on this function is exactly the same as part 1
    rows = len(maze)
    cols = len(maze[0])
    queue = deque([[(start[0],start[1]),(direction[0],direction[1]), 0]])   
    score_by_position = {(start[0],start[1]): 0}
    
    while queue:
                
        current_position_direction_score = queue.popleft()
        current_position = current_position_direction_score[0]
        current_direction = current_position_direction_score[1]
        current_score = current_position_direction_score[2]
         
        r, c = current_position
        cdr, cdc = current_direction
        steps = find_steps(current_direction)
        
        if 0 <= r < rows and 0 <= c < cols and maze[r][c] != '#':
            for dr, dc, score in steps:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] > current_score + score):
                    queue.append([(nr, nc),(dr,dc), current_score + score])
                    score_by_position[(nr, nc)] =  current_score + score
                     
    # On this run, we go from the end to the start, and we take every step that would respect the parameters of a best path
    queue = deque([[(end[0],end[1]),(1,0), score_by_position[end]],[(end[0],end[1]),(0,-1), score_by_position[end]]])   
    explored_positions = set([end])
    
    while queue:
                
        current_position_direction_score = queue.popleft()
        current_position = current_position_direction_score[0]
        current_direction = current_position_direction_score[1]
        current_score = current_position_direction_score[2]
         
        r, c = current_position
        cdr, cdc = current_direction
        steps = find_steps(current_direction)
        
        for dr, dc, score in steps:
            nr, nc = r + dr, c + dc
            # It is the check score_by_position[(nr,nc)] <= current_score that allows the step if the score of the position corresponds
            # to the best score for the position, or less, allowing to step on position that have a lower score, but are nonetheless on 
            # a path with a global score that is worse then the best path
            
            # We also do not step on positions on which we already stepped
            if (nr,nc) in score_by_position and score_by_position[(nr,nc)] <= current_score - score and (nr,nc) not in explored_positions:
                queue.append([(nr, nc),(dr,dc), current_score - score])
                new_position = (nr,nc)
                explored_positions.add(new_position)
    
    return len(explored_positions)

In [ ]:
nb_possible_positions = bfs_pathfinding_second(maze, start, (0,1), end)

print(f"Number of possible positions: {nb_possible_positions}")